### ideas
train and validate
then tale some custom images, resize and classify them

### import and parameter

In [ ]:
import shutil
import os
import tqdm as notebook_tqdm
import csv
from PIL import Image
from datasets import load_dataset
from ultralytics import YOLO
from pathlib import Path

### download from hugging face
put into /silver as the images are preprocessed: already properly named and resized

In [ ]:
CLASSES = {
    "pizza": 76,
    "spaghetti_bolognese": 90,
    "spaghetti_carbonara": 91,
}
N_SPLIT = {
    "train": 3,
    "validation": 1,
}
OUT_ROOT = Path("../data/silver")
MAKE_CSV = True
EPOCHS = 20
IMG_SIZE = 224
BATCH = 16
BASE_MODEL = "yolov8n-cls.pt"   # try yolov8s-cls.pt when you add more data
DEVICE = "cpu"                 # "auto", "cuda:0", or "cpu"

SPLIT_MAP = {"train": "train", "validation": "val"}

def map_labels():
    """Map Food-101 label ids <-> names and keep a stable class order."""
    label_names = load_dataset("ethz/food101", split="train").features["label"].names
    id_to_name = {i: n for i, n in enumerate(label_names)}
    target_pairs = [(id_to_name[food101_id], food101_id) for _, food101_id in CLASSES.items()]
    target_names = [n for n, _ in target_pairs]
    target_ids = {i for _, i in target_pairs}
    return id_to_name, target_names, target_ids

def ensure_dirs(root: Path, target_names):
    for split_food101, split_yolo in SPLIT_MAP.items():
        for cls in target_names:
            (root / split_yolo / cls).mkdir(parents=True, exist_ok=True)

def build_split(split_food101: str, OUT_ROOT: Path, target_names, target_ids, id_to_name):
    """
    Download images for the given Food-101 split and save in YOLO classification layout:
    OUT_ROOT/<train|val>/<class_name>/image.jpg
    Optionally writes a labels.csv at split root (filename,label).
    """
    split_yolo = SPLIT_MAP[split_food101]
    split_dir = OUT_ROOT / split_yolo
    split_dir.mkdir(parents=True, exist_ok=True)

    # counters by class name
    saved_counts = {cls: 0 for cls in target_names}
    target_per_class = N_SPLIT[split_food101]

    ds = load_dataset("ethz/food101", split=split_food101)

    rows = []  # optional CSV

    for idx, ex in enumerate(ds):
        lid = ex["label"]
        if lid not in target_ids:
            continue

        cls_name = id_to_name[lid]
        if saved_counts[cls_name] >= target_per_class:
            continue

        fname = f"{idx:06d}_{lid:02d}_{cls_name}.jpg"
        out_path = split_dir / cls_name / fname
        ex["image"].save(out_path)
        saved_counts[cls_name] += 1
        if MAKE_CSV:
            rows.append([f"{cls_name}/{fname}", cls_name])

        # stop early when all classes hit the quota
        if all(saved_counts[c] >= target_per_class for c in saved_counts):
            break

    # optional: CSV summary for this split
    if MAKE_CSV:
        with open(split_dir / "labels.csv", "w", newline="") as f:
            w = csv.writer(f)
            w.writerow(["filename", "label"])
            w.writerows(rows)

    print(f"[{split_food101}] saved counts:", saved_counts)

def write_dataset_yaml(root: Path, target_names):
    """
    Create dataset.yaml for YOLO classification training.
    """
    names_block = "\n".join([f"  - {n}" for n in target_names])
    yaml_text = f"""# YOLO classification dataset generated from Food-101
path: {root.resolve()}
train: train
val: val
names:
{names_block}
"""
    (root / "dataset.yaml").write_text(yaml_text)
    print(f"dataset.yaml written to: {root / 'dataset.yaml'}")
    print("Class order:", target_names)

def prepare_dataset():
    id_to_name, target_names, target_ids = map_labels()
    ensure_dirs(OUT_ROOT, target_names)
    for split in ["train", "validation"]:
        build_split(split, OUT_ROOT, target_names, target_ids, id_to_name)
    write_dataset_yaml(OUT_ROOT, target_names)
    return (OUT_ROOT / "dataset.yaml").resolve()

prepare_dataset()


In [10]:
DATASET_YAML = Path("../data/silver/")  # point to your dataset.yaml
BASE_MODEL = "yolov8n-cls.pt"
EPOCHS = 20
IMG_SIZE = 224
BATCH = 16
DEVICE = "cpu"   # or "0" if you have a GPU

def train_eval_predict():
    model = YOLO(BASE_MODEL)

    # Train
    results = model.train(
        data=str(DATASET_YAML),
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        batch=BATCH,
        device=DEVICE,
    )

    # Validate
    model.val(data=str(DATASET_YAML), imgsz=IMG_SIZE, device=DEVICE)

    # Predict on the validation folder
    model.predict(
        source=Path(DATASET_YAML).parent / "val",
        imgsz=IMG_SIZE,
        device=DEVICE,
        save=True,
    )

# direct call
train_eval_predict()


Ultralytics 8.3.201 🚀 Python-3.11.13 torch-2.8.0+cu128 CPU (Intel Xeon Platinum 8370C CPU @ 2.80GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data/silver, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train5, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True

Overriding model.yaml nc=1000 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralyti

FileNotFoundError: ../data/val does not exist